In [ ]:
import pandas as pd
import numpy as np

np.random.seed(1337)

# read processed data
data = pd.read_csv('./imdb_processed_full.csv')
data = data.iloc[np.random.permutation(len(data))]
print(data['text'].str.len().describe())

CORPUS_MAX_SIZE = 752

data['text'] = data['text'].str.slice(0, CORPUS_MAX_SIZE)

count    50000.000000
mean       858.389040
std        658.428061
min         22.000000
25%        452.000000
50%        633.000000
75%       1044.000000
max       9434.000000
Name: text, dtype: float64


In [2]:
train_size = 0.8
validation_size = 0.5

split_id = int(len(data) * train_size)

temp_train_x, test_x = data.text[:split_id], data.text[split_id:]
temp_train_y, test_y = data.label[:split_id], data.label[split_id:]

# now we have train, val and test
print('Feature Shapes:')
print('===============')
# print('Train set: {}'.format(train_x.shape))
# print('Validation set: {}'.format(val_x.shape))
print('Test set: {}'.format(test_x.shape))

Feature Shapes:
Test set: (10000,)


In [3]:
from collections import Counter
from tqdm import tqdm
tqdm.pandas()

words = temp_train_x.str.cat(sep=' ').split()

# build vocabulary
frequency_counter = Counter(words)
# sort words by the frequency they appear in the text
vocab = sorted(frequency_counter, key=frequency_counter.get, reverse=True)

# associate a number to each word in the list in ascending order
# in this way the most frequent words have lower numbers
int2word = dict(enumerate(vocab[:5000], 2))
int2word[0] = '<PAD>'
int2word[1] = '<UNK>'
word2int = {word: id for id, word in int2word.items()}
# encode words
reviews_enc = [[word2int.get(word, 1) for word in review.split()] for review in tqdm(temp_train_x.values)]

100%|██████████| 40000/40000 [00:00<00:00, 85265.29it/s]


In [4]:
# TODO: maybe float 16
features = np.zeros((len(reviews_enc), CORPUS_MAX_SIZE), dtype=int)

for i, row in enumerate(reviews_enc):
  # print(f"ROW LENGTH {len(row)}")
  # print(f"ITEM {i} COULD BE INSERTED AT {1000 - len(row)}")
  # print(f"######################")
  index = CORPUS_MAX_SIZE - len(row)
  features[i, index:] = np.array(row)[:CORPUS_MAX_SIZE]
  # features[i, :len(row)] = np.array(row)[:1000]

# make val and test set
split_val_id = int(len(temp_train_x) * validation_size)
train_x, val_x = features[:split_val_id], features[split_val_id:]
train_y, val_y = temp_train_y[:split_val_id], temp_train_y[split_val_id:]

In [5]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

# define batch size
batch_size = 128

# create tensor datasets
trainset = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y.to_numpy()))
validset = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y.to_numpy()))

# create dataloaders
trainloader = DataLoader(trainset, shuffle=True, batch_size=batch_size)
valloader = DataLoader(validset, shuffle=True, batch_size=batch_size)

In [6]:
# model architecture

class SentimentModel(nn.Module):
  def __init__(self, vocab_size, output_size, hidden_size=128, embedding_size=400, n_layers=2, dropout=0.2):
    super(SentimentModel, self).__init__()
    # embedding layer is useful to map input into vector representation
    self.embedding = nn.Embedding(vocab_size, embedding_size)
    # LSTM layer preserved by PyTorch library
    self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers, dropout=dropout, batch_first=True)
    # dropout layer
    self.dropout = nn.Dropout(0.3)
    # Linear layer for output
    self.fc = nn.Linear(hidden_size, output_size)
    # Sigmoid layer cz we will have binary classification
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    # convert feature to long
    x = x.long()
    # map input to vector
    x = self.embedding(x)
    # pass forward to lstm
    o, _ =  self.lstm(x)
    # get last sequence output
    o = o[:, -1, :]
    # apply dropout and fully connected layer
    o = self.dropout(o)
    o = self.fc(o)
    # sigmoid
    o = self.sigmoid(o)

    return o

In [7]:
# define training device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('mps' if torch.mps.is_available() else 'cpu')
print(device)

mps


In [8]:
# model hyperparamters
vocab_size = len(word2int)
output_size = 1
embedding_size = 256
hidden_size = 512
n_layers = 2
dropout=0.25

print(f"VOCAB SIZE: {vocab_size}")

# model initialization
model = SentimentModel(vocab_size, output_size, hidden_size, embedding_size, n_layers, dropout)
print(model)

VOCAB SIZE: 5002
SentimentModel(
  (embedding): Embedding(5002, 256)
  (lstm): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.25)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [9]:
# training config
lr = 0.001
criterion = nn.BCELoss()  # we use BCELoss cz we have binary classification problem
optim = Adam(model.parameters(), lr=lr)
grad_clip = 5
epochs = 8
print_every = 1
history = {
  'train_loss': [],
  'train_acc': [],
  'val_loss': [],
  'val_acc': [],
  'epochs': epochs
}
es_limit = 5

In [ ]:
model = model.to(device)

epochloop = tqdm(range(epochs), position=0, desc="Training", leave=True)

# early stop trigger
es_trigger = 0
# setting the minimun validation loss to infinity so we are sure it will be resetted
val_loss_min = torch.inf

for e in epochloop:
  # training
  model.train()
  train_loss, train_acc = 0, 0

  for id, (feature, target) in enumerate(trainloader):
    epochloop.set_postfix_str(f"Training batch {id}/{len(trainloader)}")
    feature, target = feature.to(device), target.to(device)

    # reset the optimizer to avoid accumulation of gradients across training iterations
    optim.zero_grad()
    out = model(feature)
    predicted = torch.tensor([1 if i == True else 0 for i in out > 0.5], device=device)
    equals = predicted == target
    acc = torch.mean(equals.type(torch.FloatTensor))
    train_acc += acc.item()

    loss = criterion(out.squeeze(), target.float())
    train_loss += loss.item()
    loss.backward()

    # gradient clipping to avoid exploding gradient
    nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

    # update optimizer
    optim.step()
    
    # free memory
    del feature, target, predicted
  
  history["train_loss"].append(train_loss / len(trainloader))
  history["train_acc"].append(train_acc / len(trainloader))

  # validation
  model.eval()
  val_loss, val_acc = 0, 0

  with torch.no_grad():
    for id, (feature, target) in enumerate(valloader):
      epochloop.set_postfix_str(f"Validation batch {id}/{len(valloader)}")
      feature, target = feature.to(device), target.to(device)

      out = model(feature)

      predicted = torch.tensor([1 if i == True else 0 for i in out > 0.5], device=device)
      equals = predicted == target
      acc = torch.mean(equals.type(torch.FloatTensor))
      val_acc += acc.item()

      loss = criterion(out.squeeze(), target.float())
      val_loss += loss.item()

      del feature, target, predicted

    history["val_loss"].append(val_loss/len(valloader))
    history["val_acc"].append(val_acc/len(valloader))

  model.train()
  epochloop.set_postfix_str(f"Val loss: {val_loss / len(valloader):.3f} | Val acc: {val_acc / len(valloader):.3f}")

  if (e+1) % print_every == 0:
    epochloop.write(f"epoch {e+1}/{epochs} | val loss: {val_loss/len(valloader):.3f} val acc: {val_acc/len(valloader):.3f}")
    epochloop.update()

  # save model if validation loss decrease 
  if val_loss / len(valloader) <= val_loss_min:
    torch.save(model.state_dict(), "./sentiment_lstm.pt")
    val_loss_min = val_loss / len(valloader)
    es_trigger = 0
  else:
    epochloop.write(f"[WARNING] validation loss did not improve")
    es_trigger += 1
  
  if es_trigger >= es_limit: 
    epochloop.write(f"early stopped at epoch {e+1}")
    history["epochs"] = e+1
    break

In [10]:
# map_location=torch.device('mps')
model = model.to(device)
model.load_state_dict(torch.load("./sentiment_lstm.pt", weights_only=True, map_location=torch.device('mps')))

test_enc = [[word2int.get(word, 1) for word in review.split()] for review in tqdm(test_x.values)]

test_features = np.zeros((len(test_enc), CORPUS_MAX_SIZE), dtype=int)

for i, row in enumerate(test_enc):
  index = CORPUS_MAX_SIZE - len(row)
  test_features[i, index:] = np.array(row)[:CORPUS_MAX_SIZE]

testset = TensorDataset(torch.from_numpy(test_features), torch.from_numpy(test_y.to_numpy()))
testloader = DataLoader(testset, shuffle=True, batch_size=batch_size)

# test loop
model.eval()

# metrics
test_loss = 0
test_acc = 0

all_target = []
all_predicted = []

testloop = tqdm(testloader, leave=True, desc='Inference')
with torch.no_grad():
  for feature, target in testloop:
    feature, target = feature.to(device), target.to(device)

    out = model(feature)
    
    predicted = torch.tensor([1 if i == True else 0 for i in out > 0.5], device=device)
    equals = predicted == target
    acc = torch.mean(equals.type(torch.FloatTensor))
    test_acc += acc.item()

    loss = criterion(out.squeeze(), target.float())
    test_loss += loss.item()

    all_target.extend(target.cpu().numpy())
    all_predicted.extend(predicted.cpu().numpy())

    print(f'Accuracy: {test_acc/len(testloader):.4f}, Loss: {test_loss/len(testloader):.4f}')

Inference:   1%|▏         | 1/79 [00:01<01:22,  1.06s/it]

Accuracy: 0.0104, Loss: 0.0043


Inference:   3%|▎         | 2/79 [00:01<00:53,  1.44it/s]

Accuracy: 0.0212, Loss: 0.0085


Inference:   4%|▍         | 3/79 [00:01<00:43,  1.74it/s]

...

Accuracy: 0.7998, Loss: 0.3555


Inference:  97%|█████████▋| 77/79 [00:33<00:00,  2.33it/s]

Accuracy: 0.8106, Loss: 0.3597


Inference: 100%|██████████| 79/79 [00:34<00:00,  2.29it/s]

Accuracy: 0.8212, Loss: 0.3646
Accuracy: 0.8315, Loss: 0.3720


In [11]:
from sklearn.metrics import classification_report
print(classification_report(all_predicted, all_target))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83      5004
           1       0.83      0.83      0.83      4996

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000

